In [1]:
#make all walls entire filled if willing to do so
#preferrably all paths 1 square wide
#press n after u make the maze then click to make the goal
#then click to choose starting point for player

import pygame
import time
import random
import keyboard

pygame.init()

# displayWidth = 750
# displayHeight = 750
displayWidth = 500
displayHeight = 500
size = 10

change = 1

gameDisplay = pygame.display.set_mode((displayWidth, displayHeight))

pygame.display.set_caption("Maze")

pygame 2.0.1 (SDL 2.0.14, Python 3.9.1)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
black = (0, 0, 0)
white = (255, 255, 255)
green = (0, 255, 0)
gray = (100, 100, 100)

In [3]:
def clicked(item, event):
    mx, my = pygame.mouse.get_pos()
    if (mx > item.x and mx < item.x+item.width) and (my > item.y and my < item.y+item.height):
        if event.button == 1:
            return True
    return False

def draw():
    gameDisplay.fill(black)
    for i in tiles:
        for t in i:
            t.draw()
    pygame.display.update()

def rdraw():
    gameDisplay.fill(black)
    for i in tiles:
        for t in i:
            t.draw()
    p.draw()
    pygame.display.update()
    
def quitGame():
    pygame.QUIT
    quit()

In [4]:
class Tile(object):
    def __init__(self, x, y, width, height, i):
        self.x = x
        self.y = y
        self.width = width
        self.height = height
        self.color = black
        self.wall = False
        self.value = 0
        self.goal = False
        self.id = i
    
    def draw(self):
        pygame.draw.rect(gameDisplay, self.color, (self.x,  self.y, self.width, self.height))
    
class Player(object):
    def __init__(self, x, y):
        self.x = x
        self.y = y
        self.startx = x
        self.starty = y
        self.sleeptime = 0
        self.path = []
    
    def draw(self):
        gameDisplay.blit(player, (self.x, self.y))
    
    def die(self):
        self.draw()
        self.x = self.startx
        self.y = self.starty
    
    def right(self):
        self.x += displayWidth/10
    
    def left(self):
        self.x -= displayWidth/10
        
    def up(self):
        self.y += displayHeight/10
    
    def down(self):
        self.y -= displayHeight/10
        
    def changeWall(self, tile):
        if(tile.wall == True):
            self.die()
            self.path = []
            tile.value -= change*10
            
        if(tile.wall == False):
            tile.value += change
            self.path.append(tile)
        
        if(tile.goal == True):
            for p in self.path:
                p.value += change*3
            self.sleeptime = 0.1
            self.die()
        
    def badPath(self):
        for p in self.path:
            p.value -= change*2
        self.path = []
        self.die()

In [5]:
def playerLearn():
    px = int(p.x//(displayWidth/10))
    py = int(p.y//(displayHeight/10))
    compare = []
    
    p.path.append(tiles[py][px])
        
    #top left, bottom left, top right, bottom right, left, right, top, bottom, everywhere else
    if(px == 0 and py == 0):
        compare = [tiles[py+1][px], tiles[py][px+1]]
    elif(px == 0 and py == len(tiles)-1):
        compare = [tiles[py-1][px], tiles[py][px+1]]
    elif(px == len(tiles[0])-1 and py == 0):
        compare = [tiles[py][px-1], tiles[py+1][px]]
    elif(px == len(tiles[0])-1 and py == len(tiles)-1):
        compare = [tiles[py-1][px], tiles[py][px-1]]
    elif(px == 0):
        compare = [tiles[py-1][px], tiles[py][px+1], tiles[py+1][px]]
    elif(px == len(tiles[0])-1):
        compare = [tiles[py-1][px], tiles[py][px-1], tiles[py+1][px]]
    elif(py == 0):
        compare = [tiles[py][px-1], tiles[py+1][px], tiles[py][px+1]]
    elif(py == len(tiles)-1):
        compare = [tiles[py][px-1], tiles[py-1][px], tiles[py][px+1]]
    else:
        compare = [tiles[py-1][px], tiles[py][px-1], tiles[py+1][px], tiles[py][px+1]]
            
    if(len(p.path) > 0):
        for c in range(len(compare)):
            if(compare[c].id == p.path[-1].id):
                compare.pop(c)
                break
                
    highest = float('-inf')
    for c in compare:
        highest = max(highest, c.value)
        
    for c in compare:
        if(c.value == highest):
            p.x = c.x
            p.y = c.y
            p.changeWall(c)
            nx = int(p.x//(displayWidth/10))
            ny = int(p.y//(displayHeight/10))
            if(tiles[py][px].goal):
                break
            count = 0
            if(nx == 0 and ny == 0):
                if(tiles[ny][nx+1].wall and tiles[ny+1][nx].wall):
                    tiles[ny][nx].wall = True
                    tiles[ny][nx].color = gray
            elif(nx == 0 and ny == len(tiles)-1):
                if(tiles[ny-1][nx].wall and tiles[ny][nx+1].wall):
                    tiles[ny][nx].wall = True
                    tiles[ny][nx].color = gray
            elif(nx == len(tiles[0])-1 and ny == 0):
                if(tiles[ny][nx-1].wall and tiles[ny+1][nx].wall):
                    tiles[ny][nx].wall = True
                    tiles[ny][nx].color = gray
            elif(nx == len(tiles[0])-1 and ny == len(tiles)-1):
                if(tiles[ny-1][nx].wall and tiles[ny][nx-1].wall):
                    tiles[ny][nx].wall = True
                    tiles[ny][nx].color = gray
            elif(nx == 0  or nx == len(tiles[0])-1):
                if(tiles[ny-1][nx].wall):
                    count += 1
                if(tiles[ny+1][nx].wall):
                    count += 1
                if(p.x == 0 and tiles[ny][nx+1].wall):
                    count += 1
                if(p.x == len(tiles[0])-1 and tiles[ny][nx-1].wall):
                    count += 1
                if(count >= 2):
                    tiles[ny][nx].wall = True
                    tiles[ny][nx].color = gray
            elif(ny == 0 or ny == len(tiles)-1):
                if(tiles[ny][nx-1].wall):
                    count += 1
                if(tiles[ny][nx+1].wall):
                    count += 1
                if(p.y == 0 and tiles[ny+1][nx].wall):
                    count += 1
                if(p.y == len(tiles)-1 and tiles[ny-1][nx].wall):
                    count += 1
                if(count >= 2):
                    tiles[ny][nx].wall = True
                    tiles[ny][nx].color = gray
            else:
                if(tiles[ny-1][nx].wall):
                    count += 1
                if(tiles[ny][nx+1].wall):
                    count += 1
                if(tiles[ny+1][nx].wall):
                    count += 1
                if(tiles[ny][nx-1].wall):
                    count += 1
                if(count >= 3):
                    tiles[ny][nx].wall = True
                    tiles[ny][nx].color = gray
            return
    print("welp")

def game_start():
    start = False
    draw()
    while keyboard.is_pressed('n') == False and start == False:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False
                quitGame()
            if event.type == pygame.MOUSEBUTTONDOWN:
                x, y = pygame.mouse.get_pos()
                x = int(x//(displayWidth/10))
                y = int(y//(displayHeight/10))
                t = tiles[y][x]
                if(clicked(t, event)):
                    if(t.color == black):
                        t.color = white
                        t.wall = True
                    elif(t.color == white):
                        t.color = black
                        t.wall = False
                draw()
    while start == False:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False
                quitGame()
            if event.type == pygame.MOUSEBUTTONDOWN:
                x, y = pygame.mouse.get_pos()
                x = int(x//(displayWidth/10))
                y = int(y//(displayHeight/10))
                t = tiles[y][x]
                if(clicked(t, event)):
                    t.color = green
                    t.goal = True
                    start = True
        draw()
    while start:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False
                quitGame()
            if event.type == pygame.MOUSEBUTTONDOWN:
                x, y = pygame.mouse.get_pos()
                x = int(x//(displayWidth/10))
                y = int(y//(displayHeight/10))
                t = tiles[y][x]
                if(clicked(t, event)):
                    return t.x, t.y
        draw()
def game_loop():
    running = True
    while running:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False
                quitGame()
            
        playerLearn()
        rdraw()
        for i in p.path[:-1]:
            if(p.x == i.x and p.y == i.y):
                p.badPath()
        rdraw()
        time.sleep(p.sleeptime)

In [6]:
tiles = []
count = 0
for h in range(size):
    tiles.append([])
    for w in range(size):
        tiles[h].append(Tile(w*(displayWidth/size), h*(displayHeight/size), displayHeight/size, displayWidth/size, count))
        count += 1
        
#FLATICON.COM
player = pygame.image.load("computer.png")
player = pygame.transform.scale(player, (displayHeight//10, displayWidth//10))

x, y = game_start()
p = Player(x, y)
game_loop()